# Script de python sencillo para descargar datos de EAGLE 

#### Antes de empezar: una herramienta útil para descargar datos de EAGLE desde Python, es el paquete eagleSQLtools.

#### Repositorio del paquete: https://github.com/kyleaoman/eagleSqlTools

#### Para instalarlo, en una terminal correr el siguiente comando:

#### pip install eaglesqltools

#### Importar paquetes necesarios. Agregar paquete si hace falta

In [1]:
import matplotlib.pyplot as plt
import eagleSqlTools as sql
from astropy.io import ascii
from astropy.table import Table
import numpy as np

#### Datos para la descarga (usuario, contraseña, simulación, snapnum, fragmentos de la query)

In [2]:
# Usuario y contraseña
usr='cht015'
pwd='BH457tfj'

# Simulación y snapnum deseado
simu='RefL0100N1504'
snap=28

# Carpeta donde guardar archivo. Dejar un caracter 'vacío' para descargar los datos
# en la misma carpeta desde donde se ejecuta la notebook.
download_folder=''

# Nombre del archivo que guardará los datos descargados
filename='MDM_'+simu+'_snap_'+str(snap)+'.dat'

# Cosas para armar la query
# Tabla y Alias de la tabla desde donde quiero descargar datos
table='Subhalo'
table_alias='sub'  

# Columnas a seleccionar (agregar/quitar según sea necesario)
columns=[
         'GalaxyID','Redshift','SubGroupNumber', 'Stars_Mass', 'MassType_DM'
#          'CentreOfPotential_x','CentreOfPotential_y','CentreOfPotential_z', 
#          'SF_Mass', 'NSF_Mass', 'MassType_DM', 'MassType_BH', 
#          'HalfMassRad_Star','HalfMassRad_Gas', 'HalfMassRad_DM', 
#          'Stars_Hydrogen','Stars_Oxygen','Stars_Metallicity', 
#          'SF_Hydrogen','SF_Oxygen','SF_Metallicity', 
#          'NSF_Hydrogen','NSF_Oxygen','NSF_Metallicity', 
#          'StarFormationRate','InitialMassWeightedStellarAge' 
         ]


# De qué tabla seleccionar ('FROM' en la query SQL)
from_table=simu+'_'+table+' as '+table_alias

# Condiciones para seleccionar ('WHERE' en la query SQL)
# NO AGREGAR ACÁ EL ALIAS DE LA TABLA! Se agragará después
where_cond=[
            'SnapNum='+str(snap),
            'Stars_Mass>=1e9'
           ]

#---------------------------------------------------------------------------

# Armo la sentencia 'SELECT' a partir de la lista de columnas
select=','.join([table_alias+'.'+col for col in columns])

# Armo la sentencia 'WHERE' a paritr de la lista de condiciones
where=' and '.join(where_cond)


#### Conexión a la base de datos y descarga

In [3]:
# Conectarse a la base de datos
con = sql.connect(usr,password=pwd)

# Query en SQL
query = 'SELECT '+select+' FROM '+from_table+' WHERE '+where

# Execute query 
exquery = sql.execute_query(con, query)

# List of column names of downloaded data
colnames=(exquery.view(np.recarray).dtype.names)

# Dictionary of data
mytable={}
for name in colnames:
    mytable[name]=exquery[name]
    
# dictionary={key1:value,key2:value,....}

In [4]:
mytable

{'GalaxyID': array([       1,    13631,    21793, ..., 59943139, 59945710, 59949248]),
 'Redshift': array([2.220446e-16, 2.220446e-16, 2.220446e-16, ..., 2.220446e-16,
        2.220446e-16, 2.220446e-16], dtype=float32),
 'SubGroupNumber': array([ 42,  45,  54, ..., 345, 361, 396], dtype=int32),
 'Stars_Mass': array([5.5155962e+10, 5.2623806e+10, 4.0004215e+10, ..., 1.6723877e+09,
        1.2200613e+09, 1.8930022e+09], dtype=float32),
 'MassType_DM': array([2.8144718e+10, 3.9584874e+10, 7.5883127e+10, ..., 6.3987309e+08,
        2.9763794e+09, 5.8170278e+08], dtype=float32)}

In [ ]:
# Para ver el número de galaxias descargadas
len(mytable['GalaxyID'])

#### Guardar los datos en un archivo ascii

In [5]:
# Transformar el diccionario a una tabla ascii
data_ascii=Table(mytable)

In [6]:
data_ascii

GalaxyID,Redshift,SubGroupNumber,Stars_Mass,MassType_DM
int64,float32,int32,float32,float32
1,2.220446e-16,42,55155960000.0,28144718000.0
13631,2.220446e-16,45,52623806000.0,39584874000.0
21793,2.220446e-16,54,40004215000.0,75883130000.0
23301,2.220446e-16,61,42122772000.0,67099420000.0
24477,2.220446e-16,66,26114243000.0,98627700000.0
26263,2.220446e-16,68,36057960000.0,54844875000.0
27473,2.220446e-16,70,27008782000.0,79353954000.0
28850,2.220446e-16,71,30976877000.0,73207290000.0
29817,2.220446e-16,78,23492385000.0,86615540000.0


In [6]:
# Abrir el archivo donde se guardarán los datos
outf=open(download_folder+filename,'w')

# Escribir los datos al archivo
data_ascii.write(outf,format='csv')

# Cerrar el archivo
outf.close()

#### EoP